In [ ]:
!pip install pyreadr pandas openpyxl
!pip install --upgrade lightgbm
!pip install lifelines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 13.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=139aa70858d7c245c919d58ba9630fd97ea903c3b40fc4065d1a2f040ae21c04
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


# ***Data Preparation***

**step1**

In [ ]:
import pyreadr
import pandas as pd

in_path = "/content/IMvigor210CoreBiologies.Rdata"
out_prefix = "/content/IMvigor210_data"

result = pyreadr.read_r(in_path)

with pd.ExcelWriter(f"{out_prefix}.xlsx") as xw:
    for name, df in result.items():
        csv_path = f"{out_prefix}_{name}.csv"
        df.to_csv(csv_path, index=False)

        sheet = name[:31]
        df.to_excel(xw, sheet_name=sheet, index=False)

PyreadrError: File b'/content/IMvigor210CoreBiologies.Rdata' does not exist!

In [ ]:
#sampleID 修复
import pandas as pd

anno = pd.read_csv("/content/IMvigor210_data_annoData.csv")
expr = pd.read_csv("/content/IMvigor210_data_expreSet.csv")
pheno = pd.read_csv("/content/IMvigor210_data_phenoData.csv")

sample_ids = list(expr.columns)

pheno.insert(0, "SampleID", sample_ids)

expr_T = expr.T.copy()
expr_T.index.name = "SampleID"
expr_T.reset_index(inplace=True)

pheno.to_csv("IMvigor210_pheno_fixed.csv", index=False)
expr_T.to_csv("IMvigor210_expr_fixed.csv", index=False)

In [ ]:
print("\npheno_fixed columns:", pheno_fixed.columns.tolist()[:20])
print("\nexpr_fixed columns:", expr_fixed.columns.tolist()[:20])


step2

In [ ]:
import pandas as pd
import numpy as np

pheno_path = "/content/IMvigor210_pheno_fixed.csv"
pheno = pd.read_csv(pheno_path)

col_map = {
    "Age":        "Sample age",
    "ECOG":       "Baseline ECOG Score",
    "PDL1_IC":    "IC Level",
    "PDL1_TC":    "TC Level",
    "TMB":        "FMOne mutation burden per MB",
    "Smoking":    "Tobacco Use History",
    "time":       "os",
    "censOS":     "censOS"
}


keep_cols = ["SampleID"] + [v for v in col_map.values() if v in pheno.columns]
clin = pheno[keep_cols].copy()

rename_dict = {v: k for k, v in col_map.items() if v in clin.columns}
clin.rename(columns=rename_dict, inplace=True)

for c in ["Age", "ECOG", "TMB"]:
    if c in clin.columns:
        clin[c] = pd.to_numeric(clin[c], errors="coerce")

for c in ["Age", "ECOG", "TMB"]:
    if c in clin.columns:
        mean = clin[c].mean(skipna=True)
        std  = clin[c].std(skipna=True)
        clin[c + "_z"] = (clin[c] - mean) / std

# 构建生存标签
if "time" in clin.columns and "censOS" in clin.columns:
    clin["censOS"] = pd.to_numeric(clin["censOS"], errors="coerce").fillna(1).astype(int)
    clin["event"] = 1 - clin["censOS"]
else:
    raise ValueError("缺少 'os' 或 'censOS' 列，请检查 phenoData。")

# 类别变量清洗 One-Hot 编码
cat_cols = []
for c in ["PDL1_IC", "PDL1_TC", "Smoking"]:
    if c in clin.columns:
        cat_cols.append(c)
        clin[c] = clin[c].astype(str).str.strip().replace({"nan": np.nan})

clin_encoded = pd.get_dummies(clin, columns=cat_cols, drop_first=False, dummy_na=False)


out_path = "IMvigor210_clinical_step2_processed.csv"
clin_encoded.to_csv(out_path, index=False, encoding="utf-8")

print("样本数 × 特征数：", clin_encoded.shape)

step3

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

expre_csv = "/content/IMvigor210_expr_fixed.csv"


TPM_MIN = 1.0
FRACTION_MIN = 0.10
TOP_K = 1000

expr = pd.read_csv(expre_csv, index_col=0)
expr = expr.apply(pd.to_numeric, errors='coerce')

# log1p 变换
expr_log1p = np.log1p(expr)

# 低表达过滤
mask_keep = (expr > TPM_MIN).sum(axis=1) >= max(1, int(FRACTION_MIN * expr.shape[1]))
expr_filtered = expr_log1p.loc[mask_keep]

# 按方差排序
vars_ = expr_filtered.var(axis=1).sort_values(ascending=False)
keep_top = vars_.head(min(TOP_K, expr_filtered.shape[0])).index

expr_top1000 = expr_filtered.loc[keep_top]

out_path = "rnaseq_top1000_by_variance.csv"
expr_top1000.to_csv(out_path)
print("最终 top-1000:", expr_top1000.shape)

step4

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

clinical_path = "/content/IMvigor210_clinical_step2_processed.csv"  # Step 2 输出
rnaseq_path   = "/content/rnaseq_top1000_by_variance.csv"  # Step 3 输出

clinical = pd.read_csv(clinical_path)

# 统一成字符串
clinical["SampleID"] = clinical["SampleID"].astype(str).str.strip()

rnaseq = pd.read_csv(rnaseq_path)

rnaseq["SampleID"] = rnaseq["SampleID"].astype(str).str.strip()
rnaseq = rnaseq.set_index("SampleID")


rna_ids  = rnaseq.index.astype(str)
clin_ids = clinical["SampleID"].astype(str)

common_ids = sorted(set(rna_ids).intersection(set(clin_ids)))


# 按照 common_ids 的顺序重新排列，保证两个表一一对应
clinical_sub = (
    clinical
    .set_index("SampleID")
    .loc[common_ids]
)
rnaseq_sub = rnaseq.loc[common_ids]


y_time  = clinical_sub["time"].astype(float).values
y_event = clinical_sub["event"].astype(int).values

# 构建最终特征矩阵 X
drop_cols = [c for c in ["time", "event", "censOS"] if c in clinical_sub.columns]
clinical_features = clinical_sub.drop(columns=drop_cols)

X = pd.concat([clinical_features, rnaseq_sub], axis=1)

print(f"整合后样本数: {X.shape[0]}, 特征数: {X.shape[1]}")

# 保存整合后的特征和标签（保存在当前目录）
features_out = "IMvigor210_features_step4.csv"
labels_out   = "IMvigor210_labels_step4.csv"

X.to_csv(features_out)
pd.DataFrame({
    "SampleID": common_ids,
    "time": y_time,
    "event": y_event
}).to_csv(labels_out, index=False)

print(f"已保存整合特征到: {features_out}")
print(f"已保存标签到    : {labels_out}")

# 5 折交叉验证划分（patient-level）
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

fold_assign = pd.DataFrame({
    "SampleID": common_ids,
    "fold": -1
})

for fold_id, (_, val_idx) in enumerate(skf.split(X.values, y_event), start=1):
    val_ids = [common_ids[i] for i in val_idx]
    fold_assign.loc[fold_assign["SampleID"].isin(val_ids), "fold"] = fold_id


folds_out = "IMvigor210_cv5_folds_step4.csv"
fold_assign.to_csv(folds_out, index=False)

print(f"已保存 5 折划分到: {folds_out}")



In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 读入 step4 输出
X = pd.read_csv("IMvigor210_features_step4.csv", index_col=0)
labels = pd.read_csv("IMvigor210_labels_step4.csv")
time_os = labels["time"].values
event = labels["event"].values

# 假设前面几列是临床，后面是 RNA（你可以根据列名切）
clin_cols = [c for c in X.columns if c.startswith("Age") or c.startswith("ECOG") or "PDL1" in c or "Smoking" in c or "TMB" in c]
rna_cols  = [c for c in X.columns if c not in clin_cols]

X_clin = X[clin_cols]
X_rna  = X[rna_cols]

scaler = StandardScaler()
X_rna_scaled = scaler.fit_transform(X_rna)

pca = PCA(n_components=70)
X_rna_pca = pca.fit_transform(X_rna_scaled)

X_pca = pd.concat(
    [X_clin.reset_index(drop=True),
     pd.DataFrame(X_rna_pca, columns=[f"PC{i+1}" for i in range(X_rna_pca.shape[1])])],
    axis=1
)
X_pca.to_csv("IMvigor210_features_step4_PCA50.csv")

step5

In [ ]:
import time
import pandas as pd
import numpy as np
import lightgbm as lgb

# 读取 Step4 输出
X = pd.read_csv("IMvigor210_features_step4_PCA50.csv", index_col=0)
labels = pd.read_csv("IMvigor210_labels_step4.csv")
folds = pd.read_csv("IMvigor210_cv5_folds_step4.csv")

ids = labels["SampleID"].tolist()
time_os = labels["time"].values
event = labels["event"].values

y = np.log(time_os + 1e-6)

print(f"样本数: {X.shape[0]}, 特征数: {X.shape[1]}")

# 超参数：中等网格（9 组）
num_leaves_list = [31, 63, 127]
learning_rate_list = [0.01, 0.03, 0.05]

medium_param_list = []

for nl in num_leaves_list:
    for lr in learning_rate_list:
        medium_param_list.append({
            "num_leaves": nl,
            "learning_rate": lr,
            "min_data_in_leaf": 20,
            "max_depth": -1,
            "lambda_l1": 0.0,
            "lambda_l2": 0.1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "n_estimators": 1000,
        })

print("超参组合数量：", len(medium_param_list))  # 应为 9

# 5-fold CV 训练（计时）
oof_pred = np.zeros(len(X))
model_records = []

global_start = time.time()

for fold in range(1, 6):

    fold_start = time.time()

    val_mask = folds["fold"] == fold
    train_mask = ~val_mask

    train_idx = folds.index[train_mask]
    val_idx   = folds.index[val_mask]

    X_train = X.iloc[train_idx]
    X_val   = X.iloc[val_idx]

    y_train = y[train_idx]
    y_val   = y[val_idx]

    train_set = lgb.Dataset(X_train, label=y_train)
    val_set   = lgb.Dataset(X_val, label=y_val)

    best_rmse = np.inf
    best_model = None
    best_param = None

    # 超参搜索（9 组合）
    for params in medium_param_list:
        p = params.copy()
        p.update({
            "objective": "regression",
            "metric": "rmse",
            "verbosity": -1,
            "num_threads": 4,
        })

        model = lgb.train(
            p,
            train_set,
            valid_sets=[val_set],
            num_boost_round=5000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=200, verbose=False)
            ]
        )

        rmse = model.best_score["valid_0"]["rmse"]

        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
            best_param = p

    fold_time = time.time() - fold_start
    print(f"Fold {fold} 最佳 RMSE = {best_rmse:.4f}, 用时 {fold_time:.1f} 秒")

    model_records.append(best_param)
    oof_pred[val_idx] = best_model.predict(X_val, num_iteration=best_model.best_iteration)


# Derived Risk Score = -pred
risk_score = -oof_pred

out_df = pd.DataFrame({
    "SampleID": ids,
    "predicted_log_survival": oof_pred,
    "risk_score": risk_score,
})
out_df.to_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv",
              index=False)

pd.DataFrame(model_records).to_csv(
    "IMvigor210_step5_medium_best_params_regression.csv",
    index=False
)

step6

In [ ]:
# Step 6 – Evaluation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index

pred = pd.read_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv")
labels = pd.read_csv("IMvigor210_labels_step4.csv")

df = pred.merge(labels, on="SampleID")

risk = df["risk_score"].values
time_os = df["time"].values
event = df["event"].values

print("数据量：", len(df))


cindex = concordance_index(time_os, -risk, event)


print(f"C-index = {cindex:.4f}")

with open("IMvigor210_step6_Cindex.txt", "w") as f:
    f.write(f"C-index = {cindex:.4f}")


# 按风险中位数分 high-risk / low-risk
median_risk = np.median(risk)
group = (risk >= median_risk).astype(int)

df["risk_group"] = group

kmf = KaplanMeierFitter()

plt.figure(figsize=(7,5))

# Low risk group
kmf.fit(
    durations=df[group==0]["time"],
    event_observed=df[group==0]["event"],
    label="Low Risk"
)
kmf.plot(ci_show= False)

# High risk group
kmf.fit(
    durations=df[group==1]["time"],
    event_observed=df[group==1]["event"],
    label="High Risk"
)
kmf.plot(ci_show= False)

plt.title("Kaplan–Meier Curve by Predicted Risk Groups")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.grid(alpha=0.3)

plt.savefig("IMvigor210_step6_KM_curve.png", dpi=300)
plt.show()

# ------- log-rank test -------
results = logrank_test(
    df[group==0]["time"],
    df[group==1]["time"],
    df[group==0]["event"],
    df[group==1]["event"]
)

print(f"p-value = {results.p_value:.4e}")

with open("IMvigor210_step6_logrank_p.txt", "w") as f:
    f.write(f"log-rank p-value = {results.p_value:.4e}")

Survival Time Distribution & Event Counts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

labels = pd.read_csv("IMvigor210_labels_step4.csv")

plt.figure(figsize=(7,5), dpi=150)
plt.hist(labels["time"], bins=30, edgecolor="black", alpha=0.8)
plt.xlabel("Overall survival time (months)", fontsize=12)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Distribution of survival times", fontsize=14)
plt.tick_params(axis="both", labelsize=11)
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,5), dpi=150)
labels["event"].value_counts().sort_index().plot(kind="bar", edgecolor="black")
plt.xticks([0,1], ["Censored (0)", "Death (1)"], rotation=0, fontsize=11)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Event vs censored", fontsize=14)
plt.tight_layout()
plt.show()


Risk Score Distribution

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pred = pd.read_csv("IMvigor210_step5_lightgbm_medium_regression_predictions.csv")
labels = pd.read_csv("IMvigor210_labels_step4.csv")
df = pred.merge(labels, on="SampleID")

plt.figure(figsize=(7,5), dpi=150)
plt.hist(df["risk_score"], bins=30, edgecolor="black", alpha=0.8)
plt.xlabel("Risk score", fontsize=12)
plt.ylabel("Number of patients", fontsize=12)
plt.title("Risk score distribution", fontsize=14)
plt.tick_params(axis="both", labelsize=11)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,5), dpi=150)
df.boxplot(column="risk_score", by="event")
plt.xticks([1,2], ["Censored (0)", "Death (1)"], fontsize=11)
plt.xlabel("Event", fontsize=12)
plt.ylabel("Risk score", fontsize=12)
plt.title("Risk score distribution by event", fontsize=14)
plt.suptitle("")
plt.tight_layout()
plt.show()